#  Question-answering: trả lời câu hỏi prompt với Transformer-pipeline

In [ ]:

!pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00


In [ ]:

from huggingface_hub import notebook_login

print("Please enter your Hugging Face write token:")
notebook_login()

Please enter your Hugging Face write token:


In [ ]:
from transformers import pipeline

model_name = "distilbert-base-cased-distilled-squad"
qa_pipeline = pipeline("question-answering", model=model_name, tokenizer=model_name)

print(f"Loaded pipeline with model: {model_name}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cuda:0


Loaded pipeline with model: distilbert-base-cased-distilled-squad


In [ ]:
context = """
The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.
It is named after the engineer Gustave Eiffel, whose company designed and built the tower.
Locally nicknamed 'La dame de fer' (French for 'Iron Lady'), it was constructed from 1887 to 1889
as the centerpiece of the 1889 World's Fair.
"""

question = "Who was the engineer who built the Eiffel Tower?"

result = qa_pipeline(question=question, context=context)

print(f"Question: {question}")
print(f"Answer: '{result['answer']}'")
print(f"Score: {result['score']:.4f}")

Question: Who was the engineer who built the Eiffel Tower?
Answer: 'Gustave Eiffel'
Score: 0.9923


In [ ]:

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    DefaultDataCollator
)

In [ ]:

squad = load_dataset("squad", split="train[:5000]")

squad = squad.train_test_split(test_size=0.2)

print("Dataset loaded and split:")
print(squad)


print("\nExample from training set:")
print(squad["train"][0])

README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset loaded and split:
DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1000
    })
})

Example from training set:
{'id': '56cbd2f96d243a140015ed73', 'title': 'Frédéric_Chopin', 'context': 'At the age of 21 he settled in Paris. Thereafter, during the last 18 years of his life, he gave only some 30 public performances, preferring the more intimate atmosphere of the salon. He supported himself by selling his compositions and teaching piano, for which he was in high demand. Chopin formed a friendship with Franz Liszt and was admired by many of his musical contemporaries, including Robert Schumann. In 1835 he obtained French citizenship. After a failed engagement to Maria Wodzińska, from 1837 to 1847 he maintained an often troubled relationship with the French writer George Sand. A brief and unhapp

In [ ]:

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:

max_length = 384
doc_stride = 128

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:

tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

print("Dataset has been tokenized.")

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset has been tokenized.


In [ ]:

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

data_collator = DefaultDataCollator()

your_username = "Antoine1106"
model_name = "distilbert-squad-demo"
repo_id = f"{your_username}/{model_name}"

training_args = TrainingArguments(
    output_dir=model_name,
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=True,
    hub_model_id=repo_id,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
)

print("Trainer initialized. Ready for training.")

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainer initialized. Ready for training.


In [ ]:

print("Starting training... This will take several minutes.")
trainer.train()
print("Training complete.")

Starting training... This will take several minutes.


Epoch,Training Loss,Validation Loss
1,No log,2.169486
2,2.570900,1.584610
3,2.570900,1.500313
4,0.958300,1.571113
5,0.958300,1.693086
6,0.517300,1.752862
7,0.517300,1.853515
8,0.318200,2.004554
9,0.318200,2.045493
10,0.231000,2.062897


Training complete.


In [ ]:

print("Uploading model to the Hub...")
trainer.push_to_hub()
print(f"Model successfully pushed to: {repo_id}")

Uploading model to the Hub...


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...ad-demo/training_args.bin: 100%|##########| 5.84kB / 5.84kB            

  ...787483.9f225c571ffc.212.0: 100%|##########| 4.94kB / 4.94kB            

  ...ad-demo/model.safetensors:  13%|#2        | 33.5MB /  265MB            

Model successfully pushed to: Antoine1106/distilbert-squad-demo


In [ ]:

from transformers import pipeline
import time


your_trained_model_id = repo_id

print(f"Loading your fine-tuned model: {your_trained_model_id}")
time.sleep(5)

my_pipeline = pipeline("question-answering", model=your_trained_model_id, tokenizer=your_trained_model_id)

context = """
The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.
It is named after the engineer Gustave Eiffel, whose company designed and built the tower.
Locally nicknamed 'La dame de fer' (French for 'Iron Lady'), it was constructed from 1887 to 1889
as the centerpiece of the 1889 World's Fair.
"""

question = "Which language is 'La dame de fer'?"

result = my_pipeline(question=question, context=context)

print("\n--- Inference with YOUR model ---")
print(f"Question: {question}")
print(f"Answer: '{result['answer']}'")
print(f"Score: {result['score']:.4f}")

Loading your fine-tuned model: Antoine1106/distilbert-squad-demo


Device set to use cuda:0



--- Inference with YOUR model ---
Question: Which language is 'La dame de fer'?
Answer: 'French'
Score: 0.9198
